In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.util import random_noise
from skimage import feature
import shapely
import itertools
from collections import defaultdict
from shapely.geometry import LineString, Point
from easyocr import Reader
import cv2
from PIL import Image
import cv2
from pytesseract import pytesseract
import numpy as np
import matplotlib.patches as patches

In [ ]:
# Defining path to tesseract.exe 
pytesseract.tesseract_cmd = '/bin/tesseract'

# New text detection

In [ ]:
path_abcd = "/home/joanna/Pulpit/Bioinf 2.2/ADP/projekt/pdf_tree_converter/test_data/Dendrogram.png"
path_long = "/home/joanna/Pulpit/Bioinf 2.2/ADP/projekt/pdf_tree_converter/test_data/phylogenetics_1.jpg"
path_long2 = "/home/joanna/Pulpit/Bioinf 2.2/ADP/projekt/pdf_tree_converter/test_data/phylogram_4.png"

paths = [path_abcd, path_long, path_long2]

In [ ]:
def preprocess_boxes(results):
    boxes = []
    for result in results:
        box = result[0]
        starting_point = tuple(box[0])
        width = box[1][0] - box[0][0]
        height = box[2][1] - box[0][1]
        box = [starting_point, width, height]
        boxes.append(box)
    return boxes

In [ ]:
def plot_image_with_boxes(image, boxes, figsize=(16, 9)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(image, cmap=plt.cm.gray)
    for box in boxes:
        starting_point, width, height = box
        rect = patches.Rectangle(starting_point, width, height, 
                                 linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()
    return fig

In [ ]:
reader = Reader(lang_list=["en"])

In [ ]:
boxes

In [ ]:
image_path = paths[2]
img = cv2.imread(image_path)
img = cv2.resize(img, dsize=None, fx=2, fy=2)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh1 = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU |
                                        cv2.THRESH_BINARY_INV)
if thresh1[0,0]<200:
    thresh1 = np.where(thresh1>200, 0, 255)

In [ ]:
plt.imshow(thresh1, cmap=plt.cm.gray)

In [ ]:
mg = cv2.imread(paths[0])

In [ ]:
boxes_img = {}

for path in paths:
    img = cv2.imread(path)
    
    img = cv2.resize(img, dsize=None, fx=2, fy=2)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU |cv2.THRESH_BINARY_INV)
    if thresh1[0,0]<200:
        thresh1 = np.where(thresh1>200, 0, 255)
    
    results = reader.readtext(img, 
                              min_size=5, 
                              mag_ratio=2, 
                              slope_ths=0.2, 
                             )
    text = [i[1] for i in results]
    print(text)

    boxes = preprocess_boxes(results)
    boxes_img[path] = boxes
    
    fig = plot_image_with_boxes(thresh1, boxes, figsize=(16, 9))
    
    for box in boxes:
        (x, y), w, h = box
        img[y:(y+h),x:(x+w)] = 255
        
    fig2 = px.imshow(img)
    fig2.show()

In [ ]:
boxes

In [ ]:
import numpy as np
im = cv2.imread(paths[2])
im.shape

In [ ]:
boxes_sorted_y = sorted(boxes, key=lambda x: x[1]+x[3], reverse=True) # height
tree_box = boxes_sorted_y.pop()
boxes_labels_sorted_x = sorted(boxes_sorted_y, key=lambda x: x[0]+x[2], reverse=False)
for box in boxes_labels_sorted_x:
    im2 = img.copy()
    x, y, w, h = box
    im2[y:(y+h),x:(x+h)] = 255